In [ ]:
import os
import sys
import threading
import uvicorn

os.chdir(r"D:\github\worldbank_etl_fastapi")
if os.getcwd() not in sys.path:
    sys.path.append(os.getcwd())

threading.Thread(
    target=lambda: uvicorn.run(
        "main_api:app",
        host="127.0.0.1",
        port=8002,
        log_level="info",
        access_log=False,
        reload=False,
    ),
    daemon=True,
).start()

INFO:     Started server process [8740]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8002 (Press CTRL+C to quit)


In [8]:
import requests
from app.utils.logger import logger

BASE_URL = "http://127.0.0.1:8002"

def log_test_result(status, expected, msg):
    if status == expected:
        logger.success(f"{msg} success: expected {expected}, got {status}")
    else:
        logger.warning(f"{msg} warning: expected {expected}, got {status}")

response = requests.get(f"{BASE_URL}/countries/")
log_test_result(response.status_code, 200, "get /countries/")

bad_country = {"iso3": "US", "name": "United States", "region": "Americas"}
response = requests.post(f"{BASE_URL}/countries/", json=bad_country)
log_test_result(response.status_code, 422, "post /countries/ bad iso3")

good_country = {"iso3": "USA", "name": "United States", "region": "Americas"}
response = requests.post(f"{BASE_URL}/countries/", json=good_country)
log_test_result(response.status_code, 201, "post /countries/ valid")

response = requests.post(f"{BASE_URL}/countries/", json=good_country)
log_test_result(response.status_code, 409, "post /countries/ duplicate")

country_id = response.json().get("id", 1)
response = requests.get(f"{BASE_URL}/countries/{country_id}")
log_test_result(response.status_code, 200, f"get /countries/{country_id}")

response = requests.get(f"{BASE_URL}/countries/999999")
log_test_result(response.status_code, 404, "get /countries/999999")

iv = {"indicator_id": 1, "country_id": 1, "date": 2020, "value": 123.45}
response = requests.post(f"{BASE_URL}/indicator-values/", json=iv)
log_test_result(response.status_code, 201, "post /indicator-values/")

response = requests.post(f"{BASE_URL}/indicator-values/", json=iv)
log_test_result(response.status_code, 409, "post /indicator-values/ duplicate")

response = requests.get(f"{BASE_URL}/indicator-values/999999")
log_test_result(response.status_code, 404, "get /indicator-values/999999")

indicator = {"code": "TEST01", "name": "Test Indicator", "topic_id": 1, "source_note": "Test note"}
response = requests.post(f"{BASE_URL}/indicators/", json=indicator)
log_test_result(response.status_code, 201, "post /indicators/")

response = requests.post(f"{BASE_URL}/indicators/", json=indicator)
log_test_result(response.status_code, 409, "post /indicators/ duplicate")

topic = {"name": "Test topic"}
response = requests.post(f"{BASE_URL}/topics/", json=topic)
log_test_result(response.status_code, 201, "post /topics/")

response = requests.post(f"{BASE_URL}/topics/", json=topic)
log_test_result(response.status_code, 409, "post /topics/ duplicate")

2025-06-30 16:26:33.813 | INFO     | app.routers.countries:get_countries:20 - GET /countries called with page=1, limit=10, search=''
2025-06-30 16:26:33.819 | INFO     | app.routers.countries:get_countries:28 - Returning 10 countries
2025-06-30 16:26:33.827 | SUCCESS  | __main__:log_test_result:8 - get /countries/ success: expected 200, got 200
2025-06-30 16:26:33.840 | INFO     | app.routers.countries:create_country:43 - POST /countries called with data: iso3=US, name=United States
2025-06-30 16:26:33.850 | WARNING  | __main__:log_test_result:10 - post /countries/ bad iso3 warning: expected 422, got 409


2025-06-30 16:26:33.861 | INFO     | app.routers.countries:create_country:43 - POST /countries called with data: iso3=USA, name=United States
2025-06-30 16:26:33.874 | WARNING  | __main__:log_test_result:10 - post /countries/ valid warning: expected 201, got 409
2025-06-30 16:26:33.885 | INFO     | app.routers.countries:create_country:43 - POST /countries called with data: iso3=USA, name=United States
2025-06-30 16:26:33.896 | SUCCESS  | __main__:log_test_result:8 - post /countries/ duplicate success: expected 409, got 409
2025-06-30 16:26:33.907 | INFO     | app.routers.countries:get_country:33 - GET /countries/1 called
2025-06-30 16:26:33.912 | INFO     | app.routers.countries:get_country:38 - Country with id=1 found: Aruba
2025-06-30 16:26:33.921 | SUCCESS  | __main__:log_test_result:8 - get /countries/1 success: expected 200, got 200
2025-06-30 16:26:33.932 | INFO     | app.routers.countries:get_country:33 - GET /countries/999999 called
2025-06-30 16:26:33.937 | WARNING  | app.rout